# generating benchmark data for model comparison

## p=10

In [1]:
import pandas as pd
import toytree as tt
import numpy as np
import anndata as ad
import os
import toyplot as tp
import toyplot.svg

import benchmarks.scripts.tree_data_generation as tgen


In [2]:
# tree depth
d = 5

effect_sizes = [0.3, 0.5, 0.7, 0.9]
# number of effects
num_effects = 3
# baseline parameter scale
a_abs = 2

# sampling depth
N = 10000
# dispersion
theta = 499
# samples per group
num_samples = [5, 10, 30, 50]
reps = 20


# counter through all datasets
id = 0
dataset_path = os.path.abspath("../../../tascCODA_data/benchmarks/model_comparison/datasets/")
print(dataset_path)

/Users/johannes.ostner/Documents/PhD/tascCODA/tascCODA_data/benchmarks/model_comparison/datasets


In [3]:
# Want everything to be reproducible - set a seed at every block
np.random.seed(521)
p = 10

newick = tgen.generate_tree_levels(p, d)

tree = tt.tree(newick)
tree.draw(tip_labels_align=True, node_sizes=10, node_labels='idx')

(<toyplot.canvas.Canvas at 0x7fefb2a388b0>,
 <toytree.Render.ToytreeMark at 0x7fefb2a41820>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tbe4f255c51f348728041cfcbfc377ae3"> 10 11 12 13 14 15 16 17 0 1 2 3 4 5 6 7 8 9

In [4]:
np.random.seed(56847)
effect_nodes, effect_leaves = tgen.get_effect_nodes(
    newick,
    num_effects=num_effects,
    num_leaves=p
)

print(f"nodes: {effect_nodes}")
print(f"leaves: {effect_leaves}")

tlc = ["red" if int(i) in effect_leaves else "blue" if int(i)==p-1 else "black" for i in tree.get_node_values("idx", 1, 1)[-p:]]
tlc.reverse()
ref_nodes = [p.idx for p in tree.idx_dict[p-1].get_ancestors()][:-1]
ref_nodes.append(p-1)

canvas = tp.Canvas(width=800, height=1600)
ax0 = canvas.cartesian(bounds=(0, 700, 0, 1600), padding=0)
tree.draw(
    # tip_labels=False,
    node_sizes=[20 for i in tree.get_node_values("name", 1, 1)],
    node_labels=[x for x in tree.get_node_values("idx", 1, 1)],
    node_colors=["lightcoral" if i in effect_nodes else "lightblue" if i in ref_nodes else "lightgrey" for i in tree.get_node_values("idx", 1, 1)],
    node_labels_style={"font-size": 10},
    width=700,
    height=1600,
    node_style={"stroke": "black"},
    axes=ax0,
    tip_labels="name",
    tip_labels_colors=tlc,
)
# tp.svg.render(canvas, "./plots/paper_stuff/benchmark_tree_10.svg")

effect_nodes: [12, 4, 0]
effect_leaves: [8, 0, 4, 7]
nodes: [12, 4, 0]
leaves: [8, 0, 4, 7]


(None,
 <toytree.Render.ToytreeMark at 0x7fefb2aa0be0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="800.0px" height="1600.0px" viewBox="0 0 800.0 1600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tddc8e6fc770645faa58ce6c2223510d0"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 0 1 2 3 4 5 6 7 8 9 -4 -3 -2 -1 0 0 3 6 9

In [5]:
np.random.seed(1234)
for e in effect_sizes:
    for n in num_samples:
        for r in range(reps):

            mu_0, mu_1 = tgen.generate_mu(
                a_abs=a_abs,
                num_leaves=p,
                effect_nodes=effect_nodes,
                effect_leaves=effect_leaves,
                effect_size=e,
                newick=newick
            )

            X = pd.DataFrame({"x_0": np.repeat([0,1], n)})
            Y = np.zeros((n*2, p))
            for i in range(n):
                Y[i, :] = tgen.generate_one_sample(N, mu_0, theta)
                Y[-(i+1), :] = tgen.generate_one_sample(N, mu_1, theta)

            X = X.astype(np.float64)
            Y = Y.astype(np.float64)

            test_data = ad.AnnData(
                X=Y,
                obs=X,
                uns={
                    "tree_newick": newick,
                    "effect_nodes": effect_nodes,
                    "effect_leaves": effect_leaves,
                    "effect_size": e,
                    "num_samples": n,
                }            )

            # test_data.write_h5ad(dataset_path + f"/data_{id}")
            id += 1

/Users/johannes.ostner/opt/anaconda3/envs/scCODA_3/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


## p=30

In [6]:
# Want everything to be reproducible - set a seed at every block
np.random.seed(96)
p = 30
id = 320

newick = tgen.generate_tree_levels(p, d)

tree = tt.tree(newick)
tree.draw(tip_labels_align=True, node_sizes=10, node_labels='idx')

(<toyplot.canvas.Canvas at 0x7fefb2a6f280>,
 <toytree.Render.ToytreeMark at 0x7fefb2a97940>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="540.0px" viewBox="0 0 260.0 540.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta249715f64d745fb91fe291f99588a5b"> 30 31 32 33 34 35 36 37 38 39 40 41 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29

In [7]:
np.random.seed(76)
effect_nodes, effect_leaves = tgen.get_effect_nodes(
    newick,
    num_effects=num_effects,
    num_leaves=p
)

print(f"nodes: {effect_nodes}")
print(f"leaves: {effect_leaves}")

tlc = ["red" if int(i) in effect_leaves else "blue" if int(i)==p-1 else "black" for i in tree.get_node_values("idx", 1, 1)[-p:]]
tlc.reverse()
ref_nodes = [p.idx for p in tree.idx_dict[p-1].get_ancestors()][:-1]
ref_nodes.append(p-1)

canvas = tp.Canvas(width=800, height=1600)
ax0 = canvas.cartesian(bounds=(0, 700, 0, 1600), padding=0)
tree.draw(
    # tip_labels=False,
    node_sizes=[20 for i in tree.get_node_values("name", 1, 1)],
    node_labels=[x for x in tree.get_node_values("idx", 1, 1)],
    node_colors=["lightcoral" if i in effect_nodes else "lightblue" if i in ref_nodes else "lightgrey" for i in tree.get_node_values("idx", 1, 1)],
    node_labels_style={"font-size": 10},
    width=700,
    height=1600,
    node_style={"stroke": "black"},
    axes=ax0,
    tip_labels="name",
    tip_labels_colors=tlc,
)
# tp.svg.render(canvas, "./plots/benchmark_tree_30.svg")

effect_nodes: [33, 7, 0]
effect_leaves: [0, 7, 13, 14, 15, 16]
nodes: [33, 7, 0]
leaves: [0, 7, 13, 14, 15, 16]


(None,
 <toytree.Render.ToytreeMark at 0x7fefb2aa60a0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="800.0px" height="1600.0px" viewBox="0 0 800.0 1600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tb4b2c9c2c54340b4a82d7816e3cf1152"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 -4 -3 -2 -1 0 0 10 20 30

In [8]:
np.random.seed(1234)
for e in effect_sizes:
    for n in num_samples:
        for r in range(reps):

            mu_0, mu_1 = tgen.generate_mu(
                a_abs=a_abs,
                num_leaves=p,
                effect_nodes=effect_nodes,
                effect_leaves=effect_leaves,
                effect_size=e,
                newick=newick
            )

            X = pd.DataFrame({"x_0": np.repeat([0,1], n)})
            Y = np.zeros((n*2, p))
            for i in range(n):
                Y[i, :] = tgen.generate_one_sample(N, mu_0, theta)
                Y[-(i+1), :] = tgen.generate_one_sample(N, mu_1, theta)

            X = X.astype(np.float64)
            Y = Y.astype(np.float64)

            test_data = ad.AnnData(
                X=Y,
                obs=X,
                uns={
                    "tree_newick": newick,
                    "effect_nodes": effect_nodes,
                    "effect_leaves": effect_leaves,
                    "effect_size": e,
                    "num_samples": n,
                }
            )

            # test_data.write_h5ad(dataset_path + f"/data_{id}")
            id += 1


## p=50

In [9]:
# Want everything to be reproducible - set a seed at every block
np.random.seed(96)
p = 50
id = 640

newick = tgen.generate_tree_levels(p, d)

tree = tt.tree(newick)
tree.draw(tip_labels_align=True, node_sizes=10, node_labels='idx')

(<toyplot.canvas.Canvas at 0x7fefb298cd90>,
 <toytree.Render.ToytreeMark at 0x7fefb2b4a730>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="900.0px" viewBox="0 0 260.0 900.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc8d6a5efe134434cafbebe2c0ffc581a"> 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49

In [10]:
np.random.seed(4657)
effect_nodes, effect_leaves = tgen.get_effect_nodes(
    newick,
    num_effects=num_effects,
    num_leaves=p
)

print(f"nodes: {effect_nodes}")
print(f"leaves: {effect_leaves}")

tlc = ["red" if int(i) in effect_leaves else "blue" if int(i)==p-1 else "black" for i in tree.get_node_values("idx", 1, 1)[-p:]]
tlc.reverse()
ref_nodes = [p.idx for p in tree.idx_dict[p-1].get_ancestors()][:-1]
ref_nodes.append(p-1)

canvas = tp.Canvas(width=800, height=1600)
ax0 = canvas.cartesian(bounds=(0, 700, 0, 1600), padding=0)
tree.draw(
    # tip_labels=False,
    node_sizes=[20 for i in tree.get_node_values("name", 1, 1)],
    node_labels=[x for x in tree.get_node_values("idx", 1, 1)],
    node_colors=["lightcoral" if i in effect_nodes else "lightblue" if i in ref_nodes else "lightgrey" for i in tree.get_node_values("idx", 1, 1)],
    node_labels_style={"font-size": 10},
    width=700,
    height=1600,
    node_style={"stroke": "black"},
    axes=ax0,
    tip_labels="name",
    tip_labels_colors=tlc
)
# tp.svg.render(canvas, "./plots/benchmark_tree_50.svg")

effect_nodes: [61, 40, 5]
effect_leaves: [5, 40, 44, 45, 46, 47]
nodes: [61, 40, 5]
leaves: [5, 40, 44, 45, 46, 47]


(None,
 <toytree.Render.ToytreeMark at 0x7fefb2b78430>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="800.0px" height="1600.0px" viewBox="0 0 800.0 1600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te71d68412c284010891dfccf76249126"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 -4 -3 -2 -1 0 0 10 20 30 40 50

In [11]:
np.random.seed(1234)
for e in effect_sizes:
    for n in num_samples:
        for r in range(reps):

            mu_0, mu_1 = tgen.generate_mu(
                a_abs=a_abs,
                num_leaves=p,
                effect_nodes=effect_nodes,
                effect_leaves=effect_leaves,
                effect_size=e,
                newick=newick
            )

            X = pd.DataFrame({"x_0": np.repeat([0,1], n)})
            Y = np.zeros((n*2, p))
            for i in range(n):
                Y[i, :] = tgen.generate_one_sample(N, mu_0, theta)
                Y[-(i+1), :] = tgen.generate_one_sample(N, mu_1, theta)

            X = X.astype(np.float64)
            Y = Y.astype(np.float64)

            test_data = ad.AnnData(
                X=Y,
                obs=X,
                uns={
                    "tree_newick": newick,
                    "effect_nodes": effect_nodes,
                    "effect_leaves": effect_leaves,
                    "effect_size": e,
                    "num_samples": n,
                }            )

            # test_data.write_h5ad(dataset_path + f"/data_{id}")
            id += 1

## p=100

In [12]:
# Want everything to be reproducible - set a seed at every block
np.random.seed(96)
p = 100
id = 960

newick = tgen.generate_tree_levels(p, d)

tree = tt.tree(newick)
tree.draw(tip_labels_align=True, node_sizes=10, node_labels='idx')

(<toyplot.canvas.Canvas at 0x7fefb2bb83d0>,
 <toytree.Render.ToytreeMark at 0x7fefb2bf0160>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="1000.0px" viewBox="0 0 260.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8414fa51e1314902976cc36f2f090e15"> 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99

In [13]:
np.random.seed(6543)
effect_nodes, effect_leaves = tgen.get_effect_nodes(
    newick,
    num_effects=num_effects,
    num_leaves=p
)

print(f"nodes: {effect_nodes}")
print(f"leaves: {effect_leaves}")

tlc = ["red" if int(i) in effect_leaves else "blue" if int(i)==p-1 else "black" for i in tree.get_node_values("idx", 1, 1)[-p:]]
tlc.reverse()
ref_nodes = [p.idx for p in tree.idx_dict[p-1].get_ancestors()][:-1]
ref_nodes.append(p-1)

canvas = tp.Canvas(width=800, height=1600)
ax0 = canvas.cartesian(bounds=(0, 700, 0, 1600), padding=0)
tree.draw(
    # tip_labels=False,
    node_sizes=[15 for i in tree.get_node_values("name", 1, 1)],
    node_labels=[x for x in tree.get_node_values("idx", 1, 1)],
    node_colors=["lightcoral" if i in effect_nodes else "lightblue" if i in ref_nodes else "lightgrey" for i in tree.get_node_values("idx", 1, 1)],
    node_labels_style={"font-size": 8},
    width=700,
    height=1600,
    node_style={"stroke": "black"},
    axes=ax0,
    tip_labels="name",
    tip_labels_colors=tlc
)
# tp.svg.render(canvas, "./plots/benchmark_tree_100.svg")

effect_nodes: [113, 72, 28]
effect_leaves: [72, 55, 56, 57, 58, 28]
nodes: [113, 72, 28]
leaves: [72, 55, 56, 57, 58, 28]


(None,
 <toytree.Render.ToytreeMark at 0x7fefb2c66700>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="800.0px" height="1600.0px" viewBox="0 0 800.0 1600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t78276ea03c004bf1b2e514a2b9ae8956"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 -4 -3 -2 -1 0 0 50 100

In [14]:
np.random.seed(1234)
for e in effect_sizes:
    for n in num_samples:
        for r in range(reps):

            mu_0, mu_1 = tgen.generate_mu(
                a_abs=a_abs,
                num_leaves=p,
                effect_nodes=effect_nodes,
                effect_leaves=effect_leaves,
                effect_size=e,
                newick=newick
            )

            X = pd.DataFrame({"x_0": np.repeat([0,1], n)})
            Y = np.zeros((n*2, p))
            for i in range(n):
                Y[i, :] = tgen.generate_one_sample(N, mu_0, theta)
                Y[-(i+1), :] = tgen.generate_one_sample(N, mu_1, theta)

            X = X.astype(np.float64)
            Y = Y.astype(np.float64)

            test_data = ad.AnnData(
                X=Y,
                obs=X,
                uns={
                    "tree_newick": newick,
                    "effect_nodes": effect_nodes,
                    "effect_leaves": effect_leaves,
                    "effect_size": e,
                    "num_samples": n,
                }            )

            # test_data.write_h5ad(dataset_path + f"/data_{id}")
            id += 1